In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout

from sklearn.model_selection import GroupShuffleSplit
from sklearn.preprocessing import MinMaxScaler

In [2]:
index_names = ['unit_nr', 'time_cycles']
setting_names = ['setting_1', 'setting_2', 'setting_3']
sensor_names = ['s_{}'.format(i) for i in range(1,22)] 
col_names = index_names + setting_names + sensor_names
train = pd.read_csv(r'C:\Users\user\train_FD001.txt', sep='\s+', header=None, names=col_names)
test = pd.read_csv(r'C:\Users\user\test_FD001.txt', sep='\s+', header=None, names=col_names)
y_test = pd.read_csv(r'C:\Users\user\RUL_FD001.txt', sep='\s+', header=None, names=['RUL'])

In [3]:
def add_remaining_useful_life(df):
    grouped_by_unit = df.groupby(by="unit_nr")
    max_cycle = grouped_by_unit["time_cycles"].max()
    result_frame = df.merge(max_cycle.to_frame(name='max_cycle'), left_on='unit_nr', right_index=True)
    remaining_useful_life = result_frame["max_cycle"] - result_frame["time_cycles"]
    result_frame["RUL"] = remaining_useful_life
    result_frame = result_frame.drop("max_cycle", axis=1)
    return result_frame
  
train = add_remaining_useful_life(train)
train[index_names+['RUL']].head()

,unit_nr,time_cycles,RUL
0,1,1,191
1,1,2,190
2,1,3,189
3,1,4,188
4,1,5,187


In [4]:
X_train = train.iloc[:,0:27]
y_train = X_train.pop('RUL')
X_test = test.groupby('unit_nr').last().reset_index()
y_train = y_train.clip(upper=125)

In [5]:
X=train.iloc[:,0:26]
y=train.iloc[:,26:]

In [13]:
cols = list(X_train.columns)
model = LinearRegression()
rfe = RFE(model, 18)             
X_rfe = rfe.fit_transform(X,y)  
model.fit(X_rfe,y)              
temp = pd.Series(rfe.support_,index = cols)
selected_features_rfe = temp[temp==True].index
print(selected_features_rfe)

C:\Users\user\AppData\Local\R-MINI~1\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass n_features_to_select=18 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


Index(['unit_nr', 'time_cycles', 'setting_1', 'setting_2', 's_2', 's_3', 's_4',
       's_6', 's_7', 's_8', 's_9', 's_11', 's_12', 's_13', 's_15', 's_17',
       's_20', 's_21'],
      dtype='object')


In [25]:
columnsname = ['unit_nr', 'time_cycles', 'setting_1', 'setting_2', 's_2', 's_3', 's_4',
       's_6', 's_7', 's_8', 's_9', 's_11', 's_12', 's_13', 's_15', 's_17',
       's_20', 's_21','RUL']
sensor_names=[ 's_2', 's_3', 's_4',
       's_6', 's_7', 's_8', 's_9', 's_11', 's_12', 's_13', 's_15', 's_17',
       's_20', 's_21']

In [17]:
test=test.drop(columns= ['setting_3','s_5','s_10','s_14','s_16','s_18','s_19'] )
train=train.drop(columns= ['setting_3','s_5','s_10','s_14','s_16','s_18','s_19'] )

In [20]:
train

,unit_nr,time_cycles,setting_1,setting_2,s_1,s_2,s_3,s_4,s_6,s_7,s_8,s_9,s_11,s_12,s_13,s_15,s_17,s_20,s_21,RUL
0,1,1,-0.0007,-0.0004,518.67,641.82,1589.70,1400.60,21.61,554.36,2388.06,9046.19,47.47,521.66,2388.02,8.4195,392,39.06,23.4190,191
1,1,2,0.0019,-0.0003,518.67,642.15,1591.82,1403.14,21.61,553.75,2388.04,9044.07,47.49,522.28,2388.07,8.4318,392,39.00,23.4236,190
2,1,3,-0.0043,0.0003,518.67,642.35,1587.99,1404.20,21.61,554.26,2388.08,9052.94,47.27,522.42,2388.03,8.4178,390,38.95,23.3442,189
3,1,4,0.0007,0.0000,518.67,642.35,1582.79,1401.87,21.61,554.45,2388.11,9049.48,47.13,522.86,2388.08,8.3682,392,38.88,23.3739,188
4,1,5,-0.0019,-0.0002,518.67,642.37,1582.85,1406.22,21.61,554.00,2388.06,9055.15,47.28,522.19,2388.04,8.4294,393,38.90,23.4044,187
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20626,100,196,-0.0004,-0.0003,518.67,643.49,1597.98,1428.63,21.61,551.43,2388.19,9065.52,48.07,519.49,2388.26,8.4956,397,38.49,22.9735,4
20627,100,197,-0.0016,-0.0005,518.67,643.54,1604.50,1433.58,21.61,550.86,2388.23,9065.11,48.04,519.68,2388.22,8.5139,395,38.30,23.1594,3
20628,100,198,0.0004,0.0000,518.67,643.42,1602.46,1428.18,21.61,550.94,2388.24,9065.90,48.09,520.01,2388.24,8.5646,398,38.44,22.9333,2
20629,100,199,-0.0011,0.0003,518.67,643.23,1605.26,1426.53,21.61,550.68,2388.25,9073.72,48.39,519.67,2388.23,8.5389,395,38.29,23.0640,1


In [19]:
train.to_csv('afterfeatureselectiontrain.csv')
test.to_csv('afterfeatureselectiontest.csv')

In [21]:
X_train = train.iloc[:,0:20]
y_train = X_train.pop('RUL')
X_test = test.groupby('unit_nr').last().reset_index()
y_train = y_train.clip(upper=125)

In [28]:
def evaluate(y_true, y_hat, label='test'):
    mse = mean_squared_error(y_true, y_hat)
    rmse = np.sqrt(mse)
    variance = r2_score(y_true, y_hat)
    print('{} set RMSE:{}, R2:{}'.format(label, rmse, variance))

In [26]:

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(X_train[sensor_names])
X_train_scaled = X_train.copy()
X_train_scaled[sensor_names] = pd.DataFrame(scaler.transform(X_train[sensor_names]), columns=sensor_names)


X_test_scaled = X_test.copy()
X_test_scaled[sensor_names] = pd.DataFrame(scaler.transform(X_test[sensor_names]), columns=sensor_names, index=X_test.index)

In [33]:
lm = LinearRegression()
lm.fit(X_train, y_train)
y_hat_train = lm.predict(X_train)
evaluate(y_train, y_hat_train, 'train')

y_hat_test = lm.predict(X_test)
evaluate(y_test, y_hat_test)

train set RMSE:19.818309493219108, R2:0.7738325166282622
test set RMSE:22.703624258336596, R2:0.7015092775599675


In [34]:
svr = SVR(kernel='linear', epsilon=0.2)
svr.fit(X_train_scaled, y_train)

# predict and evaluate
y_hat_train1 = svr.predict(X_train_scaled)
evaluate(y_train, y_hat_train1, 'train')

y_hat_test1 = svr.predict(X_test_scaled)
evaluate(y_test, y_hat_test1)

train set RMSE:19.915765158979962, R2:0.7716027102725291
test set RMSE:23.178657400637615, R2:0.6888878240162819


In [35]:
gss = GroupShuffleSplit(n_splits=1, train_size=0.80, random_state=42)  

def train_val_group_split(X, y, gss, groups, print_groups=True):
    
    for idx_train, idx_val in gss.split(X, y, groups=groups):
        if print_groups:
            print('train_split_engines', train.iloc[idx_train]['unit_nr'].unique(), '\n')
            print('validate_split_engines', train.iloc[idx_val]['unit_nr'].unique(), '\n')

        X_train_split = X.iloc[idx_train].copy()
        y_train_split = y.iloc[idx_train].copy()
        X_val_split = X.iloc[idx_val].copy()
        y_val_split = y.iloc[idx_val].copy()
    return X_train_split, y_train_split, X_val_split, y_val_split

split_result = train_val_group_split(X_train, y_train, gss, train['unit_nr'])
X_train_split, y_train_clipped_split, X_val_split, y_val_clipped_split = split_result

train_split_engines [  2   3   4   6   7   8   9  10  12  14  15  16  17  18  20  21  22  24
  25  26  27  28  29  30  33  35  36  37  38  39  41  42  43  44  47  48
  49  50  51  52  53  55  56  57  58  59  60  61  62  63  64  65  66  67
  68  69  70  72  73  75  76  79  80  82  83  85  86  87  88  89  90  92
  93  94  95  96  97  98  99 100] 

validate_split_engines [ 1  5 11 13 19 23 31 32 34 40 45 46 54 71 74 77 78 81 84 91] 



In [36]:
split_result = train_val_group_split(X_train_scaled, y_train, gss, train['unit_nr'], print_groups=True)
X_train_split_scaled, y_train_clipped_split_scaled, X_val_split_scaled, y_val_clipped_split_scaled = split_result

train_split_engines [  2   3   4   6   7   8   9  10  12  14  15  16  17  18  20  21  22  24
  25  26  27  28  29  30  33  35  36  37  38  39  41  42  43  44  47  48
  49  50  51  52  53  55  56  57  58  59  60  61  62  63  64  65  66  67
  68  69  70  72  73  75  76  79  80  82  83  85  86  87  88  89  90  92
  93  94  95  96  97  98  99 100] 

validate_split_engines [ 1  5 11 13 19 23 31 32 34 40 45 46 54 71 74 77 78 81 84 91] 



In [37]:
setting_names2 = ['setting_1', 'setting_2']
train_cols = setting_names2+sensor_names
input_dim = len(train_cols)

model = Sequential()
model.add(Dense(16, input_dim=input_dim, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

In [38]:
epochs = 30

history = model.fit(X_train_split_scaled[train_cols], y_train_clipped_split_scaled,
                    validation_data=(X_val_split_scaled[train_cols], y_val_clipped_split_scaled),
                    epochs=epochs)

Epoch 1/30
518/518 [==============================] - 2s 3ms/step - loss: 2545.2346 - val_loss: 475.8618
Epoch 2/30
518/518 [==============================] - 1s 2ms/step - loss: 483.2711 - val_loss: 411.9371
Epoch 3/30
518/518 [==============================] - 1s 1ms/step - loss: 444.4091 - val_loss: 380.6695
Epoch 4/30
518/518 [==============================] - 1s 1ms/step - loss: 427.7403 - val_loss: 380.4709
Epoch 5/30
518/518 [==============================] - 1s 1ms/step - loss: 415.9675 - val_loss: 353.3701
Epoch 6/30
518/518 [==============================] - 1s 1ms/step - loss: 408.0900 - val_loss: 358.8525
Epoch 7/30
518/518 [==============================] - 1s 2ms/step - loss: 402.8192 - val_loss: 339.9945
Epoch 8/30
518/518 [==============================] - 1s 2ms/step - loss: 395.2253 - val_loss: 342.3600
Epoch 9/30
518/518 [==============================] - 1s 2ms/step - loss: 390.5505 - val_loss: 329.4393
Epoch 10/30
518/518 [==============================] - 1s 2ms/s

In [39]:
y_hat_train3 = model.predict(X_train_scaled[train_cols])
evaluate(y_train, y_hat_train3, 'train')

y_hat_test3 = model.predict(X_test_scaled[train_cols])
evaluate(y_test, y_hat_test3)

train set RMSE:18.147069843369255, R2:0.8103687209356886
test set RMSE:18.888003999499986, R2:0.7934084339169978
